In [1]:
import numpy as np
import sympy as sym
import pydae.build_cffi as db

In [2]:
params_dict = {'R_w':0.316,'G':9.81,'M':1200.0,'K_w':1.0,'C_rr':0.03,'Rho':1.225,'S_f':2.13,'C_x':0.32,'K_sign':100}


u_ini_dict = {'tau_r':0.0,'beta':0.0}  # for the initialization problem
u_run_dict = {'tau_r':0.0,'beta':0.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['nu','x_pos']    # [inductor current, PI integrator]
y_ini_list = ['omega_r','snu'] # for the initialization problem
y_run_list = ['omega_r','snu'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params_dict,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [3]:
tau_w = tau_r/K_w
f_w = tau_w/R_w
f_b = G*M*sym.sin(beta)
f_d = 0.5*Rho*S_f*C_x*nu**2*snu
f_r = C_rr*G*M*snu
p_r = tau_r*omega_r

dnu = 1/M*(f_w - f_d - f_r - f_b)
dx_pos = nu - x_pos*1e-4

eq_snu = -snu + (1/(1+sym.exp(-nu*K_sign)))*2-1
eq_omega_r =-omega_r +  nu/(K_w*R_w)



In [4]:
f_list = [dnu,dx_pos]
g_list = [eq_snu,eq_omega_r]
h_dict = {'f_w':f_w,'f_d':f_d,'f_r':f_r,'tau_r':tau_r,'p_r':p_r,'nu_kmh':3.6*nu}

sys = {'name':f"mech",
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':y_list,
       'y_run_list':y_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':h_dict
      }

sys = db.build(sys)


C:\Users\jmmau\anaconda3\lib\site-packages\sympy\matrices\repmatrix.py:98: SymPyDeprecationWarning: 

non-Expr objects in a Matrix has been deprecated since SymPy 1.9. Use
list of lists, TableForm or some other data structure instead. See
https://github.com/sympy/sympy/issues/21497 for more info.

  SymPyDeprecationWarning(


Compilation time: 8.54 s


In [10]:
sys['jac_ini']

Matrix([
[                        -1.0*C_x*Rho*S_f*nu*snu/M,       0,  0, (-C_rr*G*M - 0.5*C_x*Rho*S_f*nu**2)/M],
[                                                1, -0.0001,  0,                                     0],
[2*K_sign*exp(-K_sign*nu)/(1 + exp(-K_sign*nu))**2,       0,  0,                                    -1],
[                                      1/(K_w*R_w),       0, -1,                                     0]])